In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# NMF and SVD

![NMF and SVD](NMF1.png)

- 传统的奇异值分解方法只能对稠密矩阵进行分解，即不允许所分解矩阵有空值。因而，若采用奇异值分解，需要首先填充数值，这样造成了两个问题：
    - 其一，填充大大增加了数据量，增加了算法复杂度
    - 其二，简单粗暴的数据填充很容易造成数据失真

- 使用矩阵分解具有以下优点：
    - 比较容易编程实现，随机梯度下降方法依次迭代即可训练出模型
    - 比较低的时间和空间复杂度，高维矩阵映射为两个低维矩阵节省了存储空间，训练过程比较费时，但是可以离线完成
    - 评分预测一般在线计算，直接使用离线训练得到的参数，可以实时推荐
    - 预测的精度比较高，预测准确率要高于基于领域的协同过滤以及内容过滤等方法
    - 非常好的扩展性，很方便在用户特征向量和物品特征向量中添加其它因素

- 矩阵分解的不足主要有：
    - 模型训练比较费时。
    - 推荐结果不具有很好的可解释性，分解出来的用户和物品矩阵的每个维度无法和现实生活中的概念来解释，无法用现实概念给每个维度命名，只能理解为潜在语义空间。

- 非负矩阵分解(non-negative matrix factorization)1，或非负矩阵近似(non-negative matrix approximation)，是多变量分析和线性代数的算法。
    - 输入：非负矩阵V
    - 输出：两个非负矩阵W和H
    - 目标：V=WH
- 优点
    - 处理大规模数据更快更便捷；
    - 实现简便性、分解形式和分解结果上的可解释性，占用存储空间少。
- 缺点
    - NMF中只用一层表示隐变量，无法处理复杂学习问题；
    - NMF只约束了W和H的非负性（这是唯一先验，只要求满足这个），而没有考虑到对于该先验，H内部元素间的相关性。
- 应用领域
　　- 计算机视觉，文档聚类，化学统计学，音频信号处理，推荐系统